In [1]:
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# 1) Download + read dataset (OpenML id=31: German Credit)
data = fetch_openml(data_id=31, as_frame=True)
df = data.frame.copy()

# 2) Drop missing values (if any)
df = df.dropna()

# 3) Separate X and y
target_name = data.target_names[0] if hasattr(data, "target_names") and len(data.target_names) > 0 else "class"
if target_name not in df.columns:
    df[target_name] = data.target  # fallback if needed

y = df[target_name]
X = df.drop(columns=[target_name])

# 4) Feature selection: at least 4 numeric + at least 3 nominal
numeric_cols_all = X.select_dtypes(include=["int64", "float64", "int32", "float32"]).columns.tolist()
cat_cols_all = [c for c in X.columns if c not in numeric_cols_all]

numeric_features = numeric_cols_all[:4] if len(numeric_cols_all) >= 4 else numeric_cols_all
categorical_features = cat_cols_all[:3] if len(cat_cols_all) >= 3 else cat_cols_all

X = X[numeric_features + categorical_features]

print("Selected numeric features:", numeric_features)
print("Selected categorical features:", categorical_features)
print("Total selected features:", len(numeric_features) + len(categorical_features))

# 5) Split 80/10/10 (train/val/test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.20, random_state=0, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=0, stratify=y_temp
)

# 6) Preprocessing: scaling numeric + one-hot encode categorical
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# 7) Train KNN for different k values and evaluate on validation set
k_values = [1, 3, 5, 7, 9, 11, 15, 21]
best_k = None
best_val_acc = -1
best_model = None

for k in k_values:
    model = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("knn", KNeighborsClassifier(n_neighbors=k))
    ])
    model.fit(X_train, y_train)

    y_val_pred = model.predict(X_val)
    val_acc = accuracy_score(y_val, y_val_pred)
    print(f"k={k} | validation accuracy = {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_k = k
        best_model = model

print("\nBest k based on validation:", best_k)
print("Best validation accuracy:", round(best_val_acc, 4))

# 8) Final evaluation on test set + confusion matrix for best model
y_test_pred = best_model.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)

print("\nFINAL TEST accuracy =", round(test_acc, 4))
print("\nConfusion Matrix (best model):")
print(confusion_matrix(y_test, y_test_pred))

Selected numeric features: ['duration', 'credit_amount', 'installment_commitment', 'residence_since']
Selected categorical features: ['checking_status', 'credit_history', 'purpose']
Total selected features: 7
k=1 | validation accuracy = 0.6800
k=3 | validation accuracy = 0.6600
k=5 | validation accuracy = 0.6900
k=7 | validation accuracy = 0.7000
k=9 | validation accuracy = 0.7000
k=11 | validation accuracy = 0.7400
k=15 | validation accuracy = 0.7600
k=21 | validation accuracy = 0.7900

Best k based on validation: 21
Best validation accuracy: 0.79

FINAL TEST accuracy = 0.74

Confusion Matrix (best model):
[[ 9 21]
 [ 5 65]]
